In [8]:
import tensorflow as tf
import pandas as pd
import chess.pgn
import numpy as np
from tqdm import tqdm

In [9]:
# Load the PGN file
pgn = open('data.pgn')

In [10]:
# Read the PGN file and extract the ELO ratings and moves for each game
games = []
num_games = sum(1 for _ in open("data.pgn"))

In [11]:
for i in tqdm(range(num_games)):
    game = chess.pgn.read_game(pgn)
    if game is None:
        break
    games.append(game)

  6%|▌         | 50000/859598 [02:44<44:28, 303.40it/s]  


In [12]:
# Extract the features and labels
X = []
Y = []

In [13]:
for game in tqdm(games):
    board = game.board()
    
    if "WhiteElo" in game.headers:
        elo_white = int(game.headers["WhiteElo"])
    else:
        elo_white = 0
    if "BlackElo" in game.headers:
        elo_black = int(game.headers["BlackElo"])
    else:
        elo_black = 0


    for move in game.mainline_moves():
        # Convert the board representation to a one-hot encoded format
        fen = board.fen()
        pieces = ['p', 'r', 'n', 'b', 'q', 'k', 'P', 'R', 'N', 'B', 'Q', 'K']
        piece_to_int = dict(zip(pieces, range(12)))
        one_hot = np.zeros((8, 8, 12), dtype=np.int8)
        rows = fen.split()[0].split('/')
        for i, row in enumerate(rows):
            j = 0
            for char in row:
                if char.isdigit():
                    j += int(char)
                else:
                    one_hot[i, j, piece_to_int[char]] = 1
                    j += 1
        X.append(one_hot)
        Y.append([elo_white, elo_black])
        board.push(move)

100%|██████████| 50000/50000 [05:52<00:00, 141.67it/s]


In [14]:
# Convert the features and labels to numpy arrays
X = np.array(X)
Y = np.array(Y)

In [15]:
# Split the dataset into training and testing sets
split = int(len(X) * 0.8)
X_train = X[:split]
Y_train = Y[:split]
X_test = X[split:]
Y_test = Y[split:]


In [16]:
# Normalize the feature matrix
X_train = X_train / 768
X_test = X_test / 768


In [17]:
# Define the TensorFlow model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(8, 8, 12)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(2)
])

In [18]:
def r_squared(y_true, y_pred):
    SS_res =  tf.reduce_sum(tf.square(y_true - y_pred)) 
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true))) 
    return 1 - SS_res/(SS_tot + tf.keras.backend.epsilon())

model.compile(optimizer="adam", loss="mse", metrics=[r_squared])

In [19]:
# Train the model on the training set
model.fit(X_train, Y_train, epochs=10, validation_split=0.2)

Epoch 1/10
82529/82529 [==============================] - 169s 2ms/step - loss: 945316.7500 - r_squared: -0.0569 - val_loss: 2945579.5000 - val_r_squared: -1885094163775488.0000
Epoch 2/10
82529/82529 [==============================] - 149s 2ms/step - loss: 930474.1875 - r_squared: -0.0379 - val_loss: 3071982.5000 - val_r_squared: -1965992691367936.0000
Epoch 3/10
82529/82529 [==============================] - 901s 11ms/step - loss: 929672.4375 - r_squared: -0.0368 - val_loss: 3038252.2500 - val_r_squared: -1944397998456832.0000
Epoch 4/10
82529/82529 [==============================] - 127s 2ms/step - loss: 929032.3125 - r_squared: -0.0363 - val_loss: 3072084.2500 - val_r_squared: -1966048525942784.0000
Epoch 5/10
82529/82529 [==============================] - 33887s 411ms/step - loss: 928524.0000 - r_squared: -0.0363 - val_loss: 3070165.7500 - val_r_squared: -1964827547271168.0000
Epoch 6/10
82529/82529 [==============================] - 130s 2ms/step - loss: 928207.6250 - r_squared: 

In [20]:
# Evaluate the model on the testing set
mse, mae, r2 = model.evaluate(X_test, Y_test)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("r2: ", r2)

25791/25791 [==============================] - 20s 782us/step - loss: 3078451.5000 - r_squared: -1970142435082240.0000


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
# Calculate the percentage of loss for mean squared error
loss_percent_mse = mse / (Y_test.max() - Y_test.min()) * 100
print("Mean Squared Error:", mse)

print("Percentage of loss for Mean Squared Error:", loss_percent_mse)

# Calculate the percentage of loss for mean absolute error
loss_percent_mae = mae / (Y_test.max() - Y_test.min()) * 100
print("Mean Absolute Error:", mae)
print("Percentage of loss for Mean Absolute Error:", loss_percent_mae)

In [21]:
# Convert the game moves into one-hot encoded format
board = chess.Board()
moves = ['d4', 'd5', 'Bf4', 'f6', 'Nf3', 'Nc6', 'c4', 'e5', 'dxe5', 'fxe5', 'Bxe5', 'Nxe5', 'Nxe5', 'Nf6', 'cxd5', 'Bb4+', 'Nc3', 'Bxc3+', 'bxc3', 'O-O', 'e4', 'Nxe4', 'Bc4', 'Nxf2', 'd6+', 'Kh8', 'Ng6+', 'hxg6', 'Qc2', 'Nxh1', 'O-O-O','Nf2', 'dxc7', 'Qxc7', 'Qxg6', 'Qxc4', 'Qh5+', 'Kg8', 'Qd5+', 'Qxd5', 'Rxd5']

one_hot = np.zeros((8, 8, 12), dtype=np.int8)
pieces = ['p', 'r', 'n', 'b', 'q', 'k', 'P', 'R', 'N', 'B', 'Q', 'K']
piece_to_int = dict(zip(pieces, range(12)))

for move in moves:
    board.push_san(move)
    fen = board.fen()
    rows = fen.split()[0].split('/')
    for i, row in enumerate(rows):
        j = 0
        for char in row:
            if char.isdigit():
                j += int(char)
            else:
                one_hot[i, j, piece_to_int[char]] = 1
                j += 1

# Normalize the feature matrix
one_hot = one_hot / 768

# Use the trained model to predict the Elo ratings
elo_pred = model.predict(np.array([one_hot]))
print("Predicted Elo ratings for White and Black:", elo_pred[0])


1/1 [==============================] - 0s 352ms/step
Predicted Elo ratings for White and Black: [1836.3676 1837.2216]


In [22]:
pgn = open("game.pgn")
game = chess.pgn.read_game(pgn)
board = game.board()

one_hot = np.zeros((8, 8, 12), dtype=np.int8)
pieces = ['p', 'r', 'n', 'b', 'q', 'k', 'P', 'R', 'N', 'B', 'Q', 'K']
piece_to_int = dict(zip(pieces, range(12)))

for move in game.mainline_moves():
    board.push(move)
    fen = board.fen()
    rows = fen.split()[0].split('/')
    for i, row in enumerate(rows):
        j = 0
        for char in row:
            if char.isdigit():
                j += int(char)
            else:
                one_hot[i, j, piece_to_int[char]] = 1
                j += 1

# Normalize the feature matrix
one_hot = one_hot / 768

# Use the trained model to predict the Elo ratings
elo_pred = model.predict(np.array([one_hot]))
print("Predicted Elo ratings for White and Black:", elo_pred[0])


1/1 [==============================] - 0s 15ms/step
Predicted Elo ratings for White and Black: [2512.0989 2513.0234]


In [23]:
pgn = open("alex.pgn")
game = chess.pgn.read_game(pgn)
board = game.board()

one_hot = np.zeros((8, 8, 12), dtype=np.int8)
pieces = ['p', 'r', 'n', 'b', 'q', 'k', 'P', 'R', 'N', 'B', 'Q', 'K']
piece_to_int = dict(zip(pieces, range(12)))

for move in game.mainline_moves():
    board.push(move)
    fen = board.fen()
    rows = fen.split()[0].split('/')
    for i, row in enumerate(rows):
        j = 0
        for char in row:
            if char.isdigit():
                j += int(char)
            else:
                one_hot[i, j, piece_to_int[char]] = 1
                j += 1

# Normalize the feature matrix
one_hot = one_hot / 768

# Use the trained model to predict the Elo ratings
elo_pred = model.predict(np.array([one_hot]))
print("Predicted Elo ratings for White and Black:", elo_pred[0])


1/1 [==============================] - 0s 16ms/step
Predicted Elo ratings for White and Black: [1880.0063 1880.865 ]


In [24]:
pgn = open("test.pgn")
game = chess.pgn.read_game(pgn)
board = game.board()

one_hot = np.zeros((8, 8, 12), dtype=np.int8)
pieces = ['p', 'r', 'n', 'b', 'q', 'k', 'P', 'R', 'N', 'B', 'Q', 'K']
piece_to_int = dict(zip(pieces, range(12)))

for move in game.mainline_moves():
    board.push(move)
    fen = board.fen()
    rows = fen.split()[0].split('/')
    for i, row in enumerate(rows):
        j = 0
        for char in row:
            if char.isdigit():
                j += int(char)
            else:
                one_hot[i, j, piece_to_int[char]] = 1
                j += 1

# Normalize the feature matrix
one_hot = one_hot / 768

# Use the trained model to predict the Elo ratings
elo_pred = model.predict(np.array([one_hot]))
print("Predicted Elo ratings for White and Black:", elo_pred[0])

1/1 [==============================] - 0s 19ms/step
Predicted Elo ratings for White and Black: [1823.8595 1824.7125]
